In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet50, preprocess_input
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

input_shape = 224

#change the folder name
data_folder = '2food10/'
train_gen = datagen.flow_from_directory(data_folder, target_size=(input_shape,input_shape), batch_size=512,class_mode='categorical',shuffle=True, color_mode='rgb')

Found 9999 images belonging to 10 classes.


In [0]:
def foodModel(input_sh):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(input_sh,input_sh,3), pooling=None, classes=1000)
    for layer in base_model.layers:
        layer.trainable = False
    
    X = base_model.output
    X = GlobalAveragePooling2D()(X)
    X = Dense(1024,activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(512,activation='relu')(X)
    X = Dropout(0.1)(X)
    preds = Dense(10,activation='softmax')(X)
    
    model = Model(inputs=base_model.input,outputs=preds)
    return model

In [0]:
#from keras.models import load_model

foodModel = foodModel(input_shape)
#foodModel = load_model('2food10M.h5')

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
foodModel.compile(optimizer=adam,loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
cbs = ModelCheckpoint('resnet50.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
cbs2 = EarlyStopping(monitor='loss', min_delta=0, patience=20, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
cbs3 = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

step_size_train = train_gen.n//train_gen.batch_size
foodModel.fit_generator(generator=train_gen,steps_per_epoch=step_size_train,epochs=30,verbose=1,callbacks=[cbs,cbs2,cbs3])

Epoch 1/30
312/312 [==============================] - 427s 1s/step - loss: 9.4059 - accuracy: 0.3641

Epoch 00001: loss improved from inf to 9.41728, saving model to 2food10M.h5
Epoch 2/30
312/312 [==============================] - 406s 1s/step - loss: 4.3220 - accuracy: 0.5208

Epoch 00002: loss improved from 9.41728 to 4.31883, saving model to 2food10M.h5
Epoch 3/30
312/312 [==============================] - 401s 1s/step - loss: 3.0326 - accuracy: 0.5656

Epoch 00003: loss improved from 4.31883 to 3.02945, saving model to 2food10M.h5
Epoch 4/30
312/312 [==============================] - 397s 1s/step - loss: 2.3500 - accuracy: 0.5905

Epoch 00004: loss improved from 3.02945 to 2.35133, saving model to 2food10M.h5
Epoch 5/30
312/312 [==============================] - 398s 1s/step - loss: 1.9605 - accuracy: 0.6077

Epoch 00005: loss improved from 2.35133 to 1.96258, saving model to 2food10M.h5
Epoch 6/30
312/312 [==============================] - 403s 1s/step - loss: 1.7423 - accuracy: 